In [43]:
import pandas as pd
import numpy as np
from string import punctuation
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
from natasha import DatesExtractor, MoneyExtractor
import datetime, re

import warnings
warnings.filterwarnings("ignore")

In [44]:
pd.set_option('display.max_colwidth', -1)

In [45]:
data = pd.read_csv('data.csv', error_bad_lines=False, header=0, sep=';')
data.head()

,time,text,id1,id2,name,role
0,2020-01-01 03:00:20.0422321 +03:00,"Михаил, у Вас остались дополнительные вопросы по продуктам и услугам Банка?",40711,208659.0,Сластников Олег Игоревич,оператор
1,2020-01-01 03:04:07.3632785 +03:00,Какие условия бесплатного обслуживания?,2535,208602.0,Карина Чавтур,Клиент
2,2020-01-01 03:04:46.4415198 +03:00,"Спасибо, вопросов нет",40711,208659.0,Михаил Никишин,Клиент
3,2020-01-01 03:05:35.3383325 +03:00,"Благодарю Вас за обращение в ПАО ""АК БАРС"" БАНК. Мы всегда Вам рады!",40711,208659.0,Сластников Олег Игоревич,оператор
4,2020-01-01 03:05:49.0761418 +03:00,"Мы рекомендуем Вам дебетовую карту Ак Барс Evolution, которую Вы можете заказать в мобильном приложении на экране ""Предложения"" либо на официальном сайте нашего Банка, пройдя по ссылке https://www.akbars.ru/product/individuals/cards/evolution/. Коротко о преимуществах данной карты: начисление 7% годовых на остаток от 30 000 ₽ до 100 000 ₽ и 3% — при остатке ниже 30 000 ₽ или выше 100 000 ₽, кэшбэк 1,25% на сумму покупок более 20 000 ₽ в месяц и 1% — на сумму менее или равную 20 000 ₽ в месяц, 0% комиссии при снятии наличных в банкоматах любых банков на территории РФ на общую сумму до 50 000 ₽ в месяц. С более подробными условиями по данной карте, а также с полным перечнем карт нашего Банка вы можете ознакомиться на сайте: https://www.akbars.ru/product/individuals/cards/.",2535,208602.0,Сластников Олег Игоревич,оператор


### Удаление дуплукатов и сообщений не нусущих информцию

In [46]:
data = data.drop_duplicates(subset=['text']).dropna(subset=['text']).reset_index(drop=True)

In [47]:
data.shape

(441581, 6)

In [48]:
phrases=[
        'для продолжения общения',
        'я с вами прощаюсь',
        'всего доброго',
        'всего хорошего',
        'доброй ночи',
        'хороших выходных',
        'хорошего вам дня',
        'если у вас появятся ещё вопросы',
        'будут вопросы - задавайте',
        'если возникнут вопросы',
        'у вас остались дополнительные вопросы',
        'мы всегда вам рады',
        'могу я ещё чем-то вам помочь',
        'мы всегда вам рады',
        'какой вопрос вас интересует'
        'данные направим',
        'запрос направим',
        'изображение',
        'файл',
        'уточним информацию',
        'готовлю ответ на ваш вопрос',
        'чем могу вам помочь'
    ]

In [49]:
ind=[]
for i,j in enumerate(data['text']):
    for p in phrases:
        if p in j.lower():
            ind.append(i)
            break
data = data.iloc[~data.index.isin(ind)].drop_duplicates(subset=['text']).dropna(subset=['text']).reset_index(drop=True)

In [50]:
data.shape

(429575, 6)

### Нахождение сообщений, где встречаются даты

In [51]:
dates = ['январ','феврал','март','апрел','май','июн','июл','август','сентябр','октябр','ноябр','декабр',
         'том году', 'прошлом году', 'того года', 'прошлого года', 'тот месяц', 'том месяце', 'следующем месяце', 
        'следующий месяц']

In [52]:
data['label']=np.zeros(len(data))
for i in range(len(data)):
    for date in dates:
        if date in data['text'][i].lower():
            data['label'][i] = 'date'

In [53]:
data[data['label']!=0]['text']

5         Здравствуйте! Мои расходы по карте в декабре 2019 по категории транспорт составили 6400 рублей. Начислен кэшбэк на эту сумму в размере 90 рублей. Следует отметить, что по данной категории кэшбэк составляет 10%, а именно 640рублей. Прошу Вас устранить проблему. Кудаков А.Н.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [54]:
data_with_dates = data[(data['label']!=0) & (data['role'] == 'Клиент')].iloc[:100].reset_index(drop=True)
data_with_dates

,time,text,id1,id2,name,role,label
0,2020-01-01 03:14:00.8073984 +03:00,"Здравствуйте! Мои расходы по карте в декабре 2019 по категории транспорт составили 6400 рублей. Начислен кэшбэк на эту сумму в размере 90 рублей. Следует отметить, что по данной категории кэшбэк составляет 10%, а именно 640рублей. Прошу Вас устранить проблему. Кудаков А.Н.",106319,208662.0,Андрей Кудаков,Клиент,date
1,2020-01-01 03:33:22.3950113 +03:00,Общие траты в декабре составили более 37000рублей. На топливо потрачено 16% всех платежей. Ни о каких злоупотреблениях речи не идёт. Прошу Вас решить вопрос по начислению 10%. Бензовоз имеет код 5541,106319,208662.0,Андрей Кудаков,Клиент,date
2,2020-01-01 07:05:27.3203866 +03:00,А я в этом месяце буду заново заправляться и вы мне скажете что за заправки выплачивают максимум 1000р,11772,208655.0,Дмитрий Степанов,Клиент,date
3,2020-01-01 07:17:02.0732466 +03:00,"В декабре я заправился на 8 тысяч рублей (бензин) 10% кэшбэк, а мне пришло только 630 рублей, за последнюю заправку кэшбэк не пришел",47420,208672.0,Ильдар Хуснуллин,Клиент,date
4,2020-01-01 07:35:58.8695433 +03:00,Здравствуйте. Скиньте пожалуйста расчет кэшбека за декабрь. alt-ehduard@yandex.ru,45754,208674.0,Эдуард Алтынбаев,Клиент,date
...,...,...,...,...,...,...,...
95,2020-01-01 20:32:44.1764527 +03:00,Здравствуйте! В каких дежурных офисах 2 января можно обменять доллары?,106427,208952.0,Анонимный пользователь №22024,Клиент,date
96,2020-01-01 20:46:33.9610768 +03:00,Так как возврат за декабрь был 74 рубля. А расходов по этой статье было много,5809,208955.0,Алия Шафикова,Клиент,date
97,2020-01-01 20:47:11.1932174 +03:00,"Я оплатил услуги жкх с карты generation в декабре, но мне не пришел кэшбэк. Вы же вроде 5% возвращаете",79,208956.0,Ильгам Ханнанов,Клиент,date
98,2020-01-01 20:47:40.7799169 +03:00,Добрый вечер! Просьба направить на мой эл.адрес выгрузку о начисленным бонусах по карте за декабрь,53067,208958.0,Алсу Рахимова,Клиент,date


### Лемматизация сообщений

In [55]:
tokenizer = RegexpTokenizer(r"[а-яёa-z0-9]+")
stemmer_mystem = Mystem()

def filter_text(text, tokenizer=tokenizer, stemmer_mystem=stemmer_mystem):
        text = str(text)
        tokens = tokenizer.tokenize(text.lower())
        tokens = stemmer_mystem.lemmatize(" ".join(tokens))
        tokens = [token for token in tokens if token != " " 
                and token.strip() not in punctuation]
        return " ".join(tokens)

In [56]:
for i in range(len(data_with_dates)):
    data_with_dates['text'][i] = filter_text(data_with_dates['text'][i])

### Пробная разметка с natasha

In [57]:
data_with_dates['natasha']=np.zeros(len(data_with_dates))

datesextractor = DatesExtractor()

for i in range(len(data_with_dates)):
    matches = datesextractor(data_with_dates['text'][i])
    if matches:
        data_with_dates['natasha'][i] = matches
    else:
        data_with_dates['natasha'][i] = 0

In [58]:
data_with_dates

,time,text,id1,id2,name,role,label,natasha
0,2020-01-01 03:14:00.8073984 +03:00,здравствовать мой расход по карта в декабрь 2019 по категория транспорт составлять 6400 рубль начислять кэшбэк на этот сумма в размер 90 рубль следовать отмечать что по данный категория кэшбэк составлять 10 а именно 640рублей просить вы устранять проблема кудак а н,106319,208662.0,Андрей Кудаков,Клиент,date,"(Match([MorphToken('декабрь', [36, 43), 'RU', [Form('декабрь', Grams(NOUN,accs,inan,masc,sing)), Form('декабрь', Grams(NOUN,inan,masc,nomn,sing))]), Token('2019', [44, 48), 'INT')], [36, 48)))"
1,2020-01-01 03:33:22.3950113 +03:00,общий трата в декабрь составлять более 37000рублей на топливо потратить 16 весь платеж ни о какой злоупотребление речь не идти просить вы решать вопрос по начисление 10 бензовоз иметь код 5541,106319,208662.0,Андрей Кудаков,Клиент,date,0
2,2020-01-01 07:05:27.3203866 +03:00,а я в этот месяц быть заново заправляться и вы я сказать что за заправка выплачивать максимум 1000р,11772,208655.0,Дмитрий Степанов,Клиент,date,0
3,2020-01-01 07:17:02.0732466 +03:00,в декабрь я заправляться на 8 тысяча рубль бензин 10 кэшбэк а я приходить только 630 рубль за последний заправка кэшбэк не приходить,47420,208672.0,Ильдар Хуснуллин,Клиент,date,0
4,2020-01-01 07:35:58.8695433 +03:00,здравствовать скидывать пожалуйста расчет кэшбек за декабрь alt ehduard yandex ru,45754,208674.0,Эдуард Алтынбаев,Клиент,date,0
...,...,...,...,...,...,...,...,...
95,2020-01-01 20:32:44.1764527 +03:00,здравствовать в какой дежурный офис 2 январь можно обменять доллар,106427,208952.0,Анонимный пользователь №22024,Клиент,date,"(Match([Token('2', [36, 37), 'INT'), MorphToken('январь', [38, 44), 'RU', [Form('январь', Grams(NOUN,inan,masc,nomn,sing)), Form('январь', Grams(NOUN,accs,inan,masc,sing))])], [36, 44)))"
96,2020-01-01 20:46:33.9610768 +03:00,так как возврат за декабрь быть 74 рубль а расход по этот статья быть много,5809,208955.0,Алия Шафикова,Клиент,date,0
97,2020-01-01 20:47:11.1932174 +03:00,я оплачивать услуга жкх с карта generation в декабрь но я не приходить кэшбэк вы же вроде 5 возвращать,79,208956.0,Ильгам Ханнанов,Клиент,date,0
98,2020-01-01 20:47:40.7799169 +03:00,добрый вечер просьба направлять на мой эл адрес выгрузка о начислять бонус по карта за декабрь,53067,208958.0,Алсу Рахимова,Клиент,date,0


In [59]:
datesextractor = DatesExtractor()

def rule(text):
    matches = datesextractor(text)
    par1=[]

    textn = text   
    months = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь',
        'ноябрь','декабрь']
    month_ind = [i+1 for i,j in enumerate(months) if j in textn]

    for match in matches:
        try: 
            #случай "в феврале 2022"
            par1.append([match.fact.as_json['year'],months[match.fact.as_json['month']-1], match.fact.as_json['day']])  
        except:
            #случай "в 2022 году в феврале"
            try:
                if month_ind:
                    par1.append([match.fact.as_json['year'], months[month_ind[0]-1]])
                else:
                    par1.append([match.fact.as_json['year']])
            except:
                par1.append([months[match.fact.as_json['month']-1],match.fact.as_json['day']])
        return par1

    if not par1:
        #проверяем случай "в этом году"
        if 'этот год' in textn or 'текущий год' in textn or 'нынешний год' in textn or 'в конец год' in textn:
            if month_ind:
                par1.append([datetime.datetime.now().year,months[match.fact.as_json['month']-1]])
            else:
                par1.append([datetime.datetime.now().year]) 
            return par1

        #проверяем случай "в следующем году"
        if 'следующий год' in textn or 'будущий год' in textn:
            if month_ind:
                par1.append([datetime.datetime.now().year+1,months[match.fact.as_json['month']-1]])
            else:
                par1.append([datetime.datetime.now().year+1]) 
            return par1

        #проверяем случай "в прошлом году"
        if 'прошлый год' in textn or 'тот год' in textn:
            if month_ind:
                par1.append([datetime.datetime.now().year-1,months[match.fact.as_json['month']-1]])
            else:
                par1.append([datetime.datetime.now().year-1]) 
            return par1

        #проверяем случай "в 2022"
        if re.search('[1-2][0-9]{3}', textn):
            yearn = int(re.search('[1-2][0-9]{3}', text)[0])
            if month_ind:
                par1.append([yearn,months[month_ind[0]-1]])
            else:
                par1.append([yearn]) 
            return par1

In [60]:
data_with_dates.drop(['time', 'id1', 'id2', 'role', 'label'], axis=1, inplace=True)

### Замер времени для Natasha

In [61]:
data_with_dates = data_with_dates.assign(rule_based = data_with_dates['text'])

t0 = datetime.datetime.now()

for i in range(len(data_with_dates)):
    data_with_dates['rule_based'][i] = rule(data_with_dates['rule_based'][i])
    if data_with_dates['rule_based'][i] == None:
        data_with_dates['rule_based'][i] = 0

print("Time:", (datetime.datetime.now() - t0))

Time: 0:00:00.908994


In [62]:
import numpy as np

In [63]:
data_with_dates.drop('natasha', axis=1, inplace=True)

### Загрузка Deep Pavlov и замер времени работы

In [ ]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2020-08-23 23:53:59.750 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
2020-08-23 23:54:09.873 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2020-08-23 23:54:11.556 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/dmitriy/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]


In [ ]:
data_with_dates['deeppavlov']=np.zeros(len(data_with_dates))

t0 = datetime.datetime.now()

for i in range(len(data_with_dates)):
    tokens, tags = ner_model([data_with_dates['text'][i]])
    d=[]
    for k in range(len(tags[0])):
        if 'DATE' in tags[0][k]:
            d.append(tokens[0][k])
    if d == []:
        d = 0
    data_with_dates['deeppavlov'][i] = d

print("Time:", (datetime.datetime.now() - t0))

## Сохранение сообщение с датами для последующей разметки

In [ ]:
data_with_dates.to_csv("data_with_dates.csv", index = False, sep = ';')

## Загрузка размеченного датасета с датами

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
labeled_data = pd.read_csv('labeled_dates.csv', sep = ';')
labeled_data

### Расчет f1 меры для распознавания дат Natasha и Deep Pavlov

In [ ]:
all_labels_natasha = []
all_labels_dp = []
all_labels_true = []
for row in labeled_data.itertuples():
    #true
    true_spans = [1 if str(i) in row.gt.split(',') else 0 for i in range(len(row.text.split()))]
    all_labels_true.extend(true_spans)
    #natasha
    matches = datesextractor(row.text)
    predicted_dates = []
    for match in matches: 
        predicted_date = row.text[match.span[0]:match.span[1]].split()
        predicted_dates.extend(predicted_date)
    natasha_spans = [1 if word in predicted_dates else 0 for word in row.text.split()]
    all_labels_natasha.extend(natasha_spans)
    # deep pavlov
    _, spans = ner_model([row.text])
    dp_spans = [1 if span == 'B-DATE' or span == 'I-DATE' or span == 'I-DATE' else 0 for span in spans[0]]
    all_labels_dp.extend(dp_spans)
print('Natasha f1 score: ', f1_score(all_labels_true, all_labels_natasha))
print('Deep Pavlov f1 score: ', f1_score(all_labels_true, all_labels_dp))

#### Повторная загрузка датасета, удаление дубликотов

In [ ]:
data = pd.read_csv('data.csv', error_bad_lines=False, header=0, sep=';')
data = data.drop_duplicates(subset=['text']).dropna(subset=['text']).reset_index(drop=True)

In [ ]:
ind=[]
for i,j in enumerate(data['text']):
    for p in phrases:
        if p in j.lower():
            ind.append(i)
            break
data = data.iloc[~data.index.isin(ind)].drop_duplicates(subset=['text']).dropna(subset=['text']).reset_index(drop=True)

### Нахождение сообщений с денежными суммами

In [ ]:
money_names = ['рубль', 'доллар', 'евро']
data['label']=np.zeros(len(data))
for i in range(len(data)):
    for name in money_names:
        if name in data['text'][i].lower():
            data['label'][i] = 'money'
data[data['label']!=0]['text']

In [ ]:
data_with_money = data[(data['label']!=0) & (data['role'] == 'Клиент')].iloc[:100].reset_index(drop=True)
for i in range(len(data_with_money)):
    data_with_money['text'][i] = filter_text(data_with_money['text'][i])
data_with_money

## Сохранение сообщение с датами для последующей разметки

In [ ]:
data_with_money['text'].to_csv("data_with_money.csv", index = False, sep = ';')

## Загрузка датасета с размеченными суммами

In [ ]:
labeled_data = pd.read_csv('data_with_money (1).csv', sep = ';')
labeled_data

### Расчет f1 меры для распознавания денежных сумм Natasha и Deep Pavlov

In [ ]:
all_labels_natasha = []
all_labels_dp = []
all_labels_true = []
money_extractor = MoneyExtractor()

def del_punct(text, tokenizer=tokenizer):
        text = str(text)
        tokens = tokenizer.tokenize(text.lower())
        tokens = [token for token in tokens if token != " " 
                and token.strip() not in punctuation]
        return " ".join(tokens)
    
for i in range(len(labeled_data)):
    #labeled_data['text'][i] = del_punct(labeled_data['text'][i])
    labeled_data['text'][i] = filter_text(labeled_data['text'][i])

for row in labeled_data.itertuples():
    #true
    true_spans = [1 if str(i) in row.gt.split(',') else 0 for i in range(len(row.text.split()))]
    all_labels_true.extend(true_spans)
    #natasha
    matches = money_extractor(row.text)
    predicted_dates = []
    for match in matches: 
        predicted_date = row.text[match.span[0]:match.span[1]].split()
        predicted_dates.extend(predicted_date)
    natasha_spans = [1 if word in predicted_dates else 0 for word in row.text.split()]
    all_labels_natasha.extend(natasha_spans)
    # deep pavlov
    texts, spans = ner_model([row.text])
    dp_spans = [1 if span == 'B-MONEY' or span == 'I-MONEY' or span == 'I-MONEY' else 0 for span in spans[0]]
    all_labels_dp.extend(dp_spans)
    print("Сообщение пользователя:", row.text)
    gts = [word for i, word in enumerate(row.text.split()) if str(i) in row.gt.split(',')]
    print(" ")
    print("Ground truth:", gts)
    print("Что распознала Natasha:", *predicted_dates)
    dp_res = [row.text.split()[ind] for ind,res in enumerate(dp_spans) if res == 1]
    print("Что распознал Deep Pavlov:", *dp_res)
    print('*************')
print('Natasha f1 score: ', f1_score(all_labels_true, all_labels_natasha))
print('Deep Pavlov f1 score: ', f1_score(all_labels_true, all_labels_dp))